<a href="https://colab.research.google.com/github/cs21m038/demo/blob/master/Question_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.datasets import fashion_mnist

Fetching data from fashion mnist

In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_train_val, y_train, y_train_val = train_test_split(x_train, y_train, test_size=0.10, random_state=0)

# x_train = np.array(x_train, dtype='float128').reshape([len(x_train), 784])[:1000].T
# y_train = np.array(y_train, dtype='float128')[:1000]
# x_test = np.array(x_test, dtype='float128').reshape([len(x_test), 784])[:100].T
# y_test = np.array(y_test, dtype='float128')[:100]

x_train = np.array(x_train, dtype='longdouble').reshape([len(x_train), 784]).T
y_train = np.array(y_train)
x_train_val = np.array(x_train_val, dtype='longdouble').reshape([len(x_train_val), 784]).T
y_train_val = np.array(y_train_val)
x_test = np.array(x_test, dtype='longdouble').reshape([len(x_test), 784]).T
y_test = np.array(y_test)

x_train = x_train/255.

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
(y_train.shape)[0]

54000

one hot vector

In [4]:
def one_hot(Y):
  one_hot_Y = np.zeros((Y.size, Y.max() + 1))
  one_hot_Y[np.arange(Y.size), Y] = 1
  return one_hot_Y.T

activation functions

In [5]:
def sigmoid(x):
  return 1/(1 + np.exp(-x))

def d_sigmoid(x):
  return (1 - sigmoid(x)) * sigmoid(x)

def relu(x):
  return np.maximum(x, 0)

def d_relu(x):
  return x > 0

def tanh(x):
  return (2/(1+np.exp(-2*x)))-1

def d_tanh(x):
  return 1-tanh(x)**2

# def softmax(Z):
#   A = np.exp(Z) / sum(np.exp(Z))
#   return A
def softmax(x):
    e = np.exp(x - np.max(x))
    if e.ndim == 1:
        return e / np.sum(e, axis=0)
    else: # dim = 2
        return e / np.sum(e, axis=1, keepdims=True)

def entropy(y,a):
  return -(y*np.log(a) + (1-y)*np.log(1-a))

def d_entropy(y,a):
  return (a - y)
  #/(a*(1 - a))	

implementing AddLayer class to add layers to the network

In [6]:
class AddLayer:
  def __init__(self, input_size, layer_size, act_fn,itype):
    if itype == "random":
      self.w = np.random.randn(layer_size, input_size)
      self.b = np.random.randn(layer_size,1)
      self.w = self.w.astype('longdouble')
      self.w = self.w.astype('longdouble')
    else:
      self.w = np.random.randn(layer_size, input_size)* np.sqrt(1/input_size)
      self.b = np.random.randn(layer_size,1)* np.sqrt(1/input_size)
      self.w = self.w.astype('longdouble')
      self.w = self.w.astype('longdouble')

    self.dW = np.zeros((layer_size, input_size))
    self.db = np.zeros((layer_size, 1))
    self.act = get_activation_func(act_fn)
    if(act_fn != "softmax"): self.d_act = get_d_activation_func(act_fn)

    self.prev_v_w, self.prev_v_b, self.gamma = 0, 0, 0.9
    self.m_w, self.m_b = 0, 0

  def feedForward(self, inputs):
    self.h_prev = inputs
    self.a = self.w.dot(self.h_prev) + self.b
    self.h = self.act(self.a)
    return self.h

  def set_dA(self, dA):
    self.dA = dA
  
  def refresh_dWb(self):
    self.dW = np.zeros(self.dW.shape)
    self.db = np.zeros(self.db.shape)

Adding layers to the network

In [7]:
def init_network(hidden_layers_list, act_fn, init_type):
  n_hidden_layers = len(hidden_layers_list)
  layers = []
  inputs = x_train
  for i in range(n_hidden_layers):
    #print(i , " ", inputs.shape)
    layer = AddLayer(len(inputs), hidden_layers_list[i], act_fn, init_type)
    layer.feedForward(inputs)
    inputs = layer.h
    layers.append(layer)

  outputLayer = AddLayer(len(layers[-1].h), 10, "softmax", init_type)
  output = outputLayer.feedForward(layers[-1].h)
  layers.append(outputLayer)

  return layers

forward and backward propagation functions and some utilities

In [8]:
def compute_gradient(layers, curLayer, ey, h, ind, batch_indices):
  nxtLayer = curLayer + 1
  # start = batch_indices[ind - 1]
  # end = batch_indices[ind]
  dA = []

  # for i in range(start,end):
  if curLayer == -1:
    dA.append(-(ey[:,ind] - h[:,ind]))
  else:
    dA.append((layers[nxtLayer].w.T).dot(layers[nxtLayer].dA[:,ind]) * layers[curLayer].d_act((layers[curLayer].a[:,ind])))
  return dA


def get_activation_func(ac_func):
  ac_dict = {
      "relu" : relu,
      "sigmoid" : sigmoid,
      "tanh" : tanh,
      "softmax" : softmax
  }
  return ac_dict[ac_func]

def get_d_activation_func(ac_func):
  d_ac_dict = {
      "relu" : d_relu,
      "sigmoid" : d_sigmoid,
      "tanh" : d_tanh
  }
  return d_ac_dict[ac_func]


def get_gd(gd):
  gd_dict = {
      "vanilla" : vanilla_gd,
      "momentum" : momentum_gd,
      "nesterov" : nesterov_gd,
      "stochastic" : stochastic_gd,
      "rmsprop" : rmsprop_gd,
      "adam" : adam_gd,
      "nadam" : nadam_gd
  }
  return gd_dict[gd]


def get_batch_indices(batch_size):
  m = (x_train.shape)[1]
  batch_indices = np.arange(0, m, batch_size)
  batch_indices = np.append(batch_indices, m)
  return batch_indices


def forward_propagation(layers, h):
  for layer in layers:
        h = layer.feedForward(h)
  return h

def back_propagation(layers, ey, h, gd, learning_rate, update):
#   if(gd == "vanilla" and b_sz < (y_train.shape)[0]):
#     b_sz = (y_train.shape)[0]

    grad_descent = get_gd(gd)
    outputLayer = -1
    dA = -(ey - h.flatten())
    dA = np.array([dA]).T
#     print("ey, h, dA",ey.shape, h.shape, dA.shape)
#   layers[outputLayer].set_dA(dA)
#   grad_descent(layers, outputLayer, ey, h, learning_rate, update)
    for curLayer in range((len(layers)-1), 0, -1):
        dA = grad_descent(layers, curLayer, ey, h, learning_rate, dA, update)


#gradient descent functions

In [9]:
def vanilla_gd(layers, curLayer, ey, h, b_sz, learning_rate):
  b_arr = get_batch_indices(b_sz)
  dA = compute_gradient(layers, curLayer, ey, h, 1, b_arr)
  dA = np.array(dA).T
  layers[curLayer].set_dA(dA)

  layers[curLayer].dW = 1/layers[curLayer].dA.shape[1] * layers[curLayer].dA.dot(layers[curLayer].h_prev.T)
  layers[curLayer].db = 1/layers[curLayer].dA.shape[1] * np.sum(layers[curLayer].dA)

  layers[curLayer].w = layers[curLayer].w - learning_rate * layers[curLayer].dW
  layers[curLayer].b = layers[curLayer].b - learning_rate * layers[curLayer].db



def momentum_gd(layers, curLayer, ey, h, b_sz, learning_rate):
  final_dA = []
  b_arr = get_batch_indices(b_sz)

  for i in range(1,len(b_arr)):
    start = b_arr[i-1]
    end = b_arr[i]
    for j in range(start, end):
      #feeding with new w_lookahead and b_lookahead to compute new dA
      h_previ = np.array([layers[curLayer].h_prev[:, j]]).T
      layers[curLayer].a[:, j] = (layers[curLayer].w.dot(h_previ) + layers[curLayer].b).flatten()
      layers[curLayer].h[:, j] = layers[curLayer].act(layers[curLayer].a[:, j])

      dA = compute_gradient(layers, curLayer, ey, h, j, b_arr)
      for arr in dA:
        final_dA.append(arr)
      dA = np.array(dA).T
      
      layers[curLayer].dW += dA.dot(h_previ.T)
      layers[curLayer].db += np.sum(dA)

    v_w = layers[curLayer].gamma*layers[curLayer].prev_v_w + learning_rate*layers[curLayer].dW
    v_b = layers[curLayer].gamma*layers[curLayer].prev_v_b + learning_rate*layers[curLayer].db
    layers[curLayer].refresh_dWb()

    layers[curLayer].w = layers[curLayer].w - v_w
    layers[curLayer].b = layers[curLayer].b - v_b

    layers[curLayer].prev_v_w = v_w
    layers[curLayer].prev_v_b = v_b

  final_dA = np.array(final_dA).T
  layers[curLayer].set_dA(final_dA)



def nesterov_gd(layers, curLayer, ey, h, b_sz, learning_rate):
  final_dA = []
  b_arr = get_batch_indices(b_sz)

  for i in range(1,len(b_arr)):
    start = b_arr[i-1]
    end = b_arr[i]

    v_w = layers[curLayer].gamma*layers[curLayer].prev_v_w
    v_b = layers[curLayer].gamma*layers[curLayer].prev_v_b

    w_lookAhead = layers[curLayer].w - v_w
    b_lookAhead = layers[curLayer].b - v_b

    layers[curLayer].w = w_lookAhead
    layers[curLayer].b = b_lookAhead

    for j in range(start, end):
      #feeding with new w_lookahead and b_lookahead to compute new dA
      h_previ = np.array([layers[curLayer].h_prev[:, j]]).T
      layers[curLayer].a[:, j] = (layers[curLayer].w.dot(h_previ) + layers[curLayer].b).flatten()
      layers[curLayer].h[:, j] = layers[curLayer].act(layers[curLayer].a[:, j])

      dA = compute_gradient(layers, curLayer, ey, h, j, b_arr)
      for arr in dA:
        final_dA.append(arr)
      dA = np.array(dA).T
      
      layers[curLayer].dW += dA.dot(h_previ.T)
      layers[curLayer].db += np.sum(dA)

    v_w = layers[curLayer].gamma*layers[curLayer].prev_v_w + learning_rate*layers[curLayer].dW
    v_b = layers[curLayer].gamma*layers[curLayer].prev_v_b + learning_rate*layers[curLayer].db
    layers[curLayer].refresh_dWb()
    
    layers[curLayer].w = layers[curLayer].w - v_w
    layers[curLayer].b = layers[curLayer].b - v_b

    layers[curLayer].prev_v_w = v_w
    layers[curLayer].prev_v_b = v_b

  final_dA = np.array(final_dA).T
  layers[curLayer].set_dA(final_dA)



def stochastic_gd(layers, curLayer, ey, h, learning_rate, dA, update):
    h_previ = np.array(layers[curLayer].h_prev)
#     layers[curLayer].a = (layers[curLayer].w.dot(h_previ) + layers[curLayer].b).flatten()
#     layers[curLayer].h = layers[curLayer].act(layers[curLayer].a)
    
    layers[curLayer].dW += dA.dot(h_previ.T)
    layers[curLayer].db += dA
    
#     print("h_prev of cur",layers[curLayer].h_prev.shape)
#     print("dA next", dA.shape)
    dh = (layers[curLayer].w.T).dot(dA)
    dA = dh * layers[curLayer-1].d_act((layers[curLayer-1].a))
#     print("dWcur",layers[curLayer].dW.shape)
    if update != 0:
        layers[curLayer].w = layers[curLayer].w - learning_rate * layers[curLayer].dW
        layers[curLayer].b = layers[curLayer].b - learning_rate * layers[curLayer].db
        layers[curLayer].refresh_dWb()

#   final_dA = np.array(final_dA).T
#     layers[curLayer].set_dA(dA)
    return dA


def rmsprop_gd(layers, curLayer, ey, h, b_sz, eta):
  final_dA = []
  b_arr = get_batch_indices(b_sz)
  beta = 0.9
  eps = 1e-8

  for i in range(1,len(b_arr)):
    start = b_arr[i-1]
    end = b_arr[i]
    for j in range(start, end):
      #feeding with new w_lookahead and b_lookahead to compute new dA
      h_previ = np.array([layers[curLayer].h_prev[:, j]]).T
      layers[curLayer].a[:, j] = (layers[curLayer].w.dot(h_previ) + layers[curLayer].b).flatten()
      layers[curLayer].h[:, j] = layers[curLayer].act(layers[curLayer].a[:, j])

      dA = compute_gradient(layers, curLayer, ey, h, j, b_arr)
      for arr in dA:
        final_dA.append(arr)
      dA = np.array(dA).T
      
      layers[curLayer].dW += dA.dot(h_previ.T)
      layers[curLayer].db += np.sum(dA)

    v_w = beta*layers[curLayer].prev_v_w + (1 - beta)*(layers[curLayer].dW**2)
    v_b = beta*layers[curLayer].prev_v_b + beta*(layers[curLayer].db**2)


    layers[curLayer].w = layers[curLayer].w - (eta/(np.sqrt(v_w + eps)))*layers[curLayer].dW
    layers[curLayer].b = layers[curLayer].b - (eta/(np.sqrt(v_b + eps)))*layers[curLayer].db
    layers[curLayer].refresh_dWb()
    
    layers[curLayer].prev_v_w = v_w
    layers[curLayer].prev_v_b = v_b

  final_dA = np.array(final_dA).T
  layers[curLayer].set_dA(final_dA)


def adam_gd(layers, curLayer, ey, h, b_sz, eta):
  final_dA = []
  b_arr = get_batch_indices(b_sz)
  beta1, beta2, eps, t = 0.9, 0.99, 1e-8, 0

  for i in range(1,len(b_arr)):
    start = b_arr[i-1]
    end = b_arr[i]
    for j in range(start, end):
      #feeding with new w_lookahead and b_lookahead to compute new dA
      h_previ = np.array([layers[curLayer].h_prev[:, j]]).T
      layers[curLayer].a[:, j] = (layers[curLayer].w.dot(h_previ) + layers[curLayer].b).flatten()
      layers[curLayer].h[:, j] = layers[curLayer].act(layers[curLayer].a[:, j])

      dA = compute_gradient(layers, curLayer, ey, h, j, b_arr)
      for arr in dA:
        final_dA.append(arr)
      dA = np.array(dA).T
      
      layers[curLayer].dW += dA.dot(h_previ.T)
      layers[curLayer].db += np.sum(dA)

    m_w = beta1*layers[curLayer].m_w + (1 - beta1)*(layers[curLayer].dW)
    m_b = beta2*layers[curLayer].m_b + (1 - beta1)*(layers[curLayer].db)

    v_w = beta2*layers[curLayer].prev_v_w + (1 - beta2)*(layers[curLayer].dW**2)
    v_b = beta2*layers[curLayer].prev_v_b + (1 - beta2)*(layers[curLayer].db**2)

    t = t + 1
    m_w_hat = m_w/(1-np.power(beta1,t))
    m_b_hat = m_b/(1-np.power(beta1,t))

    v_w_hat = v_w/(1-np.power(beta2,t))
    v_b_hat = v_b/(1-np.power(beta2,t))

    #updates 
    layers[curLayer].w = layers[curLayer].w - (eta/(np.sqrt(v_w_hat + eps)))*m_w_hat
    layers[curLayer].b = layers[curLayer].b - (eta/(np.sqrt(v_b_hat + eps)))*m_b_hat
    layers[curLayer].refresh_dWb()

    layers[curLayer].prev_v_w = v_w
    layers[curLayer].prev_v_b = v_b

    layers[curLayer].m_w = m_w
    layers[curLayer].m_b = m_b

  final_dA = np.array(final_dA).T
  layers[curLayer].set_dA(final_dA)


def nadam_gd(layers, curLayer, ey, h, b_sz, eta):
  final_dA = []
  b_arr = get_batch_indices(b_sz)
  beta1, beta2, eps, t = 0.9, 0.99, 1e-8, 0

  for i in range(1,len(b_arr)):
    start = b_arr[i-1]
    end = b_arr[i]
    for j in range(start, end):
      #feeding with new w_lookahead and b_lookahead to compute new dA
      h_previ = np.array([layers[curLayer].h_prev[:, j]]).T
      layers[curLayer].a[:, j] = (layers[curLayer].w.dot(h_previ) + layers[curLayer].b).flatten()
      layers[curLayer].h[:, j] = layers[curLayer].act(layers[curLayer].a[:, j])

      dA = compute_gradient(layers, curLayer, ey, h, j, b_arr)
      for arr in dA:
        final_dA.append(arr)
      dA = np.array(dA).T
      
      layers[curLayer].dW += dA.dot(h_previ.T)
      layers[curLayer].db += np.sum(dA)

    m_w = beta1*layers[curLayer].m_w + (1 - beta1)*(layers[curLayer].dW)
    m_b = beta2*layers[curLayer].m_b + (1 - beta1)*(layers[curLayer].db)

    v_w = beta2*layers[curLayer].prev_v_w + (1 - beta2)*(layers[curLayer].dW**2)
    v_b = beta2*layers[curLayer].prev_v_b + (1 - beta2)*(layers[curLayer].db**2)

    t = t + 1
    m_w_hat = m_w/(1-np.power(beta1,t))
    m_b_hat = m_b/(1-np.power(beta1,t))

    m_w_lookahead = beta1*m_w_hat - ((1 - beta1)*layers[curLayer].dW/(1-np.power(beta1,t)))
    m_b_lookahead = beta1*m_b_hat - ((1 - beta1)*layers[curLayer].db/(1-np.power(beta1,t)))

    v_w_hat = v_w/(1-np.power(beta2,t))
    v_b_hat = v_b/(1-np.power(beta2,t))

    #updates 
    layers[curLayer].w = layers[curLayer].w - (eta/(np.sqrt(v_w_hat + eps)))*m_w_lookahead
    layers[curLayer].b = layers[curLayer].b - (eta/(np.sqrt(v_b_hat + eps)))*m_b_lookahead
    layers[curLayer].refresh_dWb()
    
    layers[curLayer].prev_v_w = v_w
    layers[curLayer].prev_v_b = v_b

    layers[curLayer].m_w = m_w
    layers[curLayer].m_b = m_b

  final_dA = np.array(final_dA).T
  layers[curLayer].set_dA(final_dA)

In [10]:
#@title
# x = np.array([[2,3,4,5,1],[6,7,8,9,10],[21,22,23,24,25]])

# dotx = []
# for i in range(x.shape[1]):
#   dotx.append(x[:,i])
#   print(dotx)
# dotx = np.array(dotx).T
# print(dotx)

measuring functions

In [11]:
def get_predictions(A2):
    return np.argmax(A2, 0)
def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

# RUNNING EPOCHS

In [14]:
m = x_train.shape[1]
epochs = 5 # no of iterations
hidden_layers_list = [128,128]
batch_size =  16
activation = "relu"
init_type = "random"
layers = init_network(hidden_layers_list, activation, init_type)
learning_rate = 0.001
cost = []
ey = one_hot(y_train)
for epoch in range(epochs):
    b_arr = get_batch_indices(batch_size)
    for i in range(1,len(b_arr)):
        start = b_arr[i-1]
        end = b_arr[i]
        for j in range(start, end):
            x_data = np.array([x_train[:,j]]).T
            y_cap = forward_propagation(layers, x_data)
            flag = ((j == start and j != 0) or j+1 == m)
            back_propagation(layers, ey[:,j], y_cap, "stochastic", learning_rate, flag)

#     ey = one_hot(y_train)
    loss = 1/m * np.sum(entropy(ey, y_cap)) #this h is y_hat now 
    cost.append(loss)

    if epoch % 1 == 0 or epoch == epochs-1:
        y_cap = forward_propagation(layers, x_train)
        print("Iteration: ", epoch, "\n")
        print("accuracy on train data = ", get_accuracy(get_predictions(y_cap), y_train))

        y_test_pred = x_train_val
        for layer in layers:
            y_test_pred = layer.feedForward(y_test_pred)
        print("accuracy on validation  = ", get_accuracy(get_predictions(y_test_pred), y_train_val))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide


KeyboardInterrupt: ignored

# TESTING DATA WITH IMAGE SHOW

In [ ]:
y_test_pred = x_test
for layer in layers:
  y_test_pred = layer.feedForward(y_test_pred)

print("accuracy on test data : ", get_accuracy(get_predictions(y_test_pred), y_test))

def test_prediction(index):
  current_image = x_train[:, index, None]
  prediction = np.argmax(y_test_pred[:,index], 0)
  label = y_train[index]
  print(y_test_pred[:,index])
  print("Prediction: ", prediction)
  print("Label: ", label)
  
  current_image = current_image.reshape((28, 28)) * 255
  plt.gray()
  plt.imshow(current_image, interpolation='nearest')
  plt.show()

In [ ]:
test_prediction(7)